In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Sequential
from tensorflow.keras.applications import VGG16

In [9]:
batch_Size = 32
image_Size = 196
epochs = 5
channels = 3

In [10]:
data_dir = 'C:/Semester 4/Penglihatan citra dan computer vision/Fire_nonfire/dataset/fire_dataset'


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_Size , image_Size ),
  batch_size=batch_Size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_Size , image_Size ),
  batch_size=batch_Size)




Found 999 files belonging to 2 classes.
Using 800 files for training.
Found 999 files belonging to 2 classes.
Using 199 files for validation.


In [14]:
scaling = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(196, 196),
    layers.experimental.preprocessing.Rescaling(1./255.)
])

In [11]:
labels = train_ds.class_names
labels

['fire_images', 'non_fire_images']

In [15]:
model = Sequential([
    scaling,
    layers.Conv2D(64, kernel_size = (3,3), activation='relu', input_shape=(batch_Size, image_Size, image_Size, channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.1),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.1),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [17]:
model.compile(    optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy']
    )
history = model.fit(train_ds,
                       batch_size=batch_Size,
                       validation_data=val_ds,
                       verbose=1,
                       epochs=10)

Epoch 1/10
25/25 [==============================] - 5s 153ms/step - loss: 0.2160 - accuracy: 0.9162 - val_loss: 0.1312 - val_accuracy: 0.9698
Epoch 2/10
25/25 [==============================] - 5s 146ms/step - loss: 0.1095 - accuracy: 0.9613 - val_loss: 0.2568 - val_accuracy: 0.9296
Epoch 3/10
25/25 [==============================] - 5s 148ms/step - loss: 0.1553 - accuracy: 0.9388 - val_loss: 0.1361 - val_accuracy: 0.9698
Epoch 4/10
25/25 [==============================] - 5s 143ms/step - loss: 0.0932 - accuracy: 0.9712 - val_loss: 0.1242 - val_accuracy: 0.9447
Epoch 5/10
25/25 [==============================] - 5s 146ms/step - loss: 0.1209 - accuracy: 0.9688 - val_loss: 0.1516 - val_accuracy: 0.9246
Epoch 6/10
25/25 [==============================] - 5s 144ms/step - loss: 0.1244 - accuracy: 0.9638 - val_loss: 0.1097 - val_accuracy: 0.9598
Epoch 7/10
25/25 [==============================] - 5s 157ms/step - loss: 0.0577 - accuracy: 0.9762 - val_loss: 0.1123 - val_accuracy: 0.9698
Epoch 

In [24]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(196, 196, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False


In [25]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history2=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    batch_size=batch_Size
)

Epoch 1/10
25/25 [==============================] - 6s 172ms/step - loss: 0.0214 - accuracy: 0.9925 - val_loss: 0.2963 - val_accuracy: 0.9397
Epoch 2/10
25/25 [==============================] - 5s 166ms/step - loss: 0.0110 - accuracy: 0.9975 - val_loss: 0.3208 - val_accuracy: 0.9296
Epoch 3/10
25/25 [==============================] - 5s 162ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.3167 - val_accuracy: 0.9296
Epoch 4/10
25/25 [==============================] - 5s 172ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.3502 - val_accuracy: 0.9347
Epoch 5/10
25/25 [==============================] - 5s 171ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.3651 - val_accuracy: 0.9347
Epoch 6/10
25/25 [==============================] - 5s 171ms/step - loss: 9.9291e-04 - accuracy: 1.0000 - val_loss: 0.3662 - val_accuracy: 0.9347
Epoch 7/10
25/25 [==============================] - 5s 171ms/step - loss: 8.1028e-04 - accuracy: 1.0000 - val_loss: 0.3658 - val_accuracy: 0.934

In [32]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(196, 196, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False


In [33]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [34]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history3=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    batch_size=batch_Size
)

Epoch 1/10
25/25 [==============================] - 6s 156ms/step - loss: 0.8208 - accuracy: 0.8213 - val_loss: 0.3629 - val_accuracy: 0.9246
Epoch 2/10
25/25 [==============================] - 5s 168ms/step - loss: 0.1955 - accuracy: 0.9438 - val_loss: 0.1599 - val_accuracy: 0.9497
Epoch 3/10
25/25 [==============================] - 5s 164ms/step - loss: 0.0803 - accuracy: 0.9663 - val_loss: 0.1350 - val_accuracy: 0.9548
Epoch 4/10
25/25 [==============================] - 5s 170ms/step - loss: 0.0510 - accuracy: 0.9800 - val_loss: 0.1187 - val_accuracy: 0.9447
Epoch 5/10
25/25 [==============================] - 5s 173ms/step - loss: 0.0348 - accuracy: 0.9937 - val_loss: 0.1205 - val_accuracy: 0.9447
Epoch 6/10
25/25 [==============================] - 8s 245ms/step - loss: 0.0271 - accuracy: 0.9937 - val_loss: 0.1278 - val_accuracy: 0.9447
Epoch 7/10
25/25 [==============================] - 6s 176ms/step - loss: 0.0201 - accuracy: 0.9950 - val_loss: 0.1252 - val_accuracy: 0.9447
Epoch 

In [35]:
from tensorflow.keras.applications import InceptionV3


In [38]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(196, 196, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [39]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [40]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history4=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    batch_size=batch_Size
)

Epoch 1/10
25/25 [==============================] - 9s 196ms/step - loss: 4.7586 - accuracy: 0.7538 - val_loss: 1.4485 - val_accuracy: 0.7940
Epoch 2/10
25/25 [==============================] - 5s 147ms/step - loss: 1.1356 - accuracy: 0.8100 - val_loss: 1.4519 - val_accuracy: 0.7990
Epoch 3/10
25/25 [==============================] - 5s 150ms/step - loss: 0.4797 - accuracy: 0.8712 - val_loss: 0.9916 - val_accuracy: 0.7789
Epoch 4/10
25/25 [==============================] - 5s 148ms/step - loss: 0.4794 - accuracy: 0.8413 - val_loss: 1.2437 - val_accuracy: 0.8141
Epoch 5/10
25/25 [==============================] - 5s 145ms/step - loss: 0.4467 - accuracy: 0.8737 - val_loss: 0.9762 - val_accuracy: 0.7688
Epoch 6/10
25/25 [==============================] - 5s 144ms/step - loss: 0.3814 - accuracy: 0.8775 - val_loss: 0.9649 - val_accuracy: 0.8090
Epoch 7/10
25/25 [==============================] - 5s 146ms/step - loss: 0.3028 - accuracy: 0.9137 - val_loss: 1.2295 - val_accuracy: 0.7286
Epoch 

In [41]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(196, 196, 3))
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


94765736/94765736 [==============================] - 17s 0us/step


In [42]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history5=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    batch_size=batch_Size
)

Epoch 1/10
25/25 [==============================] - 7s 172ms/step - loss: 0.3107 - accuracy: 0.8763 - val_loss: 0.0909 - val_accuracy: 0.9749
Epoch 2/10
25/25 [==============================] - 5s 150ms/step - loss: 0.0616 - accuracy: 0.9775 - val_loss: 0.0776 - val_accuracy: 0.9799
Epoch 3/10
25/25 [==============================] - 5s 147ms/step - loss: 0.0428 - accuracy: 0.9862 - val_loss: 0.0851 - val_accuracy: 0.9698
Epoch 4/10
25/25 [==============================] - 5s 144ms/step - loss: 0.0350 - accuracy: 0.9887 - val_loss: 0.0771 - val_accuracy: 0.9749
Epoch 5/10
25/25 [==============================] - 5s 145ms/step - loss: 0.0260 - accuracy: 0.9950 - val_loss: 0.0802 - val_accuracy: 0.9749
Epoch 6/10
25/25 [==============================] - 5s 149ms/step - loss: 0.0181 - accuracy: 0.9962 - val_loss: 0.0916 - val_accuracy: 0.9698
Epoch 7/10
25/25 [==============================] - 5s 145ms/step - loss: 0.0137 - accuracy: 0.9987 - val_loss: 0.0945 - val_accuracy: 0.9648
Epoch 

In [43]:
model.save('fire_nonfire_model_restnet.h5')